In [1]:
%cd ..

/Users/albafranco/Documents/Python/visa-scoring


In [2]:
import pandas as pd
from src.read_pdfs_v2 import process_applications
from src.preprocess_table_inmi import preprocess_applications_df
from src.model_training import train_model

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/paddle/utils/cpp_extension/extension_utils.py:711: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)


In [3]:
df_grant_list = pd.read_csv('data_headers/header_Grant.csv')
df_ref_list = pd.read_csv('data_headers/header_Refusal.csv')

headers_list = list(set(df_grant_list.super_header.tolist() + df_ref_list.super_header.tolist()))

type_case = 'Grant'
##list_headers_g = scan_folder_and_save_csv('data_all/Info Students/'+type_case, 'header_'+type_case+'.csv', check_folders = 500)
df_g = process_applications('data_all/Info Students/'+type_case,headers_list, check_folders = None)

type_case = 'Refusal'
#list_headers_r = scan_folder_and_save_csv('data_all/Info Students/'+type_case, 'header_'+type_case+'.csv',check_folders = 300)
df_r = process_applications('data_all/Info Students/'+type_case,headers_list, check_folders = None)

Starting application processing...


Cases:  58%|█████▊    | 5382/9315 [17:34<07:14,  9.04it/s]  

MuPDF error: syntax error: cannot load shading function (2233 0 R)

MuPDF error: syntax error: cannot load shading function (2233 0 R)

MuPDF error: syntax error: cannot load shading function (2233 0 R)

MuPDF error: syntax error: cannot load shading function (2233 0 R)

MuPDF error: syntax error: cannot load shading function (2233 0 R)

MuPDF error: syntax error: cannot load shading function (2233 0 R)

MuPDF error: syntax error: cannot load shading function (2233 0 R)

MuPDF error: syntax error: cannot load shading function (2233 0 R)

MuPDF error: syntax error: cannot load shading function (2233 0 R)

MuPDF error: syntax error: cannot load shading function (2233 0 R)

MuPDF error: syntax error: cannot load shading function (2233 0 R)

MuPDF error: syntax error: cannot load shading function (2233 0 R)

MuPDF error: syntax error: cannot load shading function (2233 0 R)

MuPDF error: syntax error: cannot load shading function (2233 0 R)

MuPDF error: syntax error: cannot load shading f

Cases: 100%|██████████| 9315/9315 [29:27<00:00,  5.27it/s]


Finished processing. Total applications processed: 6741
Starting application processing...


Cases: 100%|██████████| 740/740 [02:23<00:00,  5.17it/s]


Finished processing. Total applications processed: 595


In [ ]:
df_g['type_case'] = 'granted'
df_r['type_case'] = 'refused'
df_cases = pd.concat([df_g,df_r])
df_cases.to_csv('data_scripts/data_onevisa_s.csv', index=True)
df_cases.shape

(7336, 4662)

In [3]:
df_cases = pd.read_csv('data_scripts/data_onevisa_s.csv').set_index('sample_id')

In [4]:
df_preprocess = preprocess_applications_df(df_cases)

Step 1: Dropping all-null rows...
Remaining rows: 7336
Step 2: Dropping rows where all key counts are zero...
Remaining rows after drop: 6749
Step 3: Keeping rows with non-null 'current_location_ac_cl'...
Remaining rows: 6742
Step 4: Dropping columns that are fully NaN...
Remaining columns: 607
Step 5: Unifying mutually exclusive columns...
Step 6: Summary of unique values and NaNs...
Step 7: Cleaning and converting 'generated' field...
Step 8: Dropping object columns with >80% unique values...
Remaining columns: 594
Step 9: Dropping date columns with >80% missing values...
Remaining columns after dropping sparse dates: 551
Step 10: Filtering incorrect date entries...
Step 11: Dropping explicit date ranges...
Step 12: Converting remaining date columns to time deltas...
Step 13: Adding 'days_from_today' field...
Step 15: Removing postal code, mobile phone and passport number columns...
No columns with mixed data types found.
Final shape of the preprocessed dataset: (6742, 507)


In [5]:
df_preprocess = df_preprocess.reset_index()

In [10]:
model_xgb, df_pred_xgb = train_model(df_preprocess, model_type ='xgboost',
                                     target_col='type_case', threshold=0.25)


Preparing data...
Encoding categorical columns with OrdinalEncoder for XGBoost...
Splitting data into train/test sets...
Training XGBoostClassifier...
Evaluating model (threshold = 0.5)...
Accuracy:  0.933
Precision: 0.945
Recall:    0.983

Full classification report:
               precision    recall  f1-score   support

     granted       0.95      0.98      0.96      1235
     refused       0.68      0.39      0.49       114

    accuracy                           0.93      1349
   macro avg       0.81      0.68      0.73      1349
weighted avg       0.92      0.93      0.92      1349

Median predicted probabilities:
Prob Refused of refused:  %.3f 0.36937124
Prob Granted of refused:  %.3f 0.63062876
Prob Refused of granted:  %.3f 0.021869104
Prob Granted of granted:  %.3f 0.9781309
Evaluating with custom threshold = 0.25...
Thresholded Accuracy:  0.929
Thresholded Precision: 0.961
Thresholded Recall:    0.961

Full classification report (with threshold):
               precision   